In [0]:
epochs = 50

# পর্ব ৭ - ফেডারেটেড ডাটাসেটের সাথে ফেডারেটেড লার্নিং

এখানে আমরা ফেডারেটেড ডেটাসেট ব্যবহারের জন্য একটি নতুন সরঞ্জাম পরিচিত করি। আমরা পাইটর্চ ডেটাসেট ক্লাসের মতো ব্যবহার করার উদ্দেশ্যে একটি `FederatedDataset` ক্লাস তৈরি করেছি, এবং এটি ফেডারেটেড ডাটালোডার `FederatedDataLoader` কে দেয়া হয়েছে যা কিনা ফেডারেটেড ভঙ্গিতে এটি ইটারেট করবে।


লেখক:
- Andrew Trask - টুইটারঃ [@iamtrask](https://twitter.com/iamtrask)
- Théo Ryffel - গিটহাবঃ [@LaRiffle](https://github.com/LaRiffle)

অনুবাদক:
- সায়ন্তন দাস - গিটহাবঃ [@ucalyptus](https://github.com/ucalyptus)
- Mir Mohammad Jaber - টুইটারঃ [@jabertuhin](https://twitter.com/jabertuhin)


আমরা স্যান্ডবক্সটি ব্যবহার করি যা আমরা শেষ পাঠে আবিষ্কার করেছি

In [0]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), verbose=False)

এরপর ডাটাসেটের খোঁজ করি

In [0]:
boston_data = grid.search("#boston", "#data")
boston_target = grid.search("#boston", "#target")

আমরা একটি মডেল এবং একটি অপ্টিমাইজার লোড করি

In [0]:
n_features = boston_data['alice'][0].shape[1]
n_targets = 1

model = th.nn.Linear(n_features, n_targets)

এখানে আমরা প্রাপ্ত ডেটাকে `FederatedDataset`-এ  কাস্ট করেছি। সেসব কর্মীদের দেখি যারা আংশিক ডাটা ধরে আছে -

In [0]:
# Cast the result in BaseDatasets
datasets = []
for worker in boston_data.keys():
    dataset = sy.BaseDataset(boston_data[worker][0], boston_target[worker][0])
    datasets.append(dataset)

# Build the FederatedDataset object
dataset = sy.FederatedDataset(datasets)
print(dataset.workers)
optimizers = {}
for worker in dataset.workers:
    optimizers[worker] = th.optim.Adam(params=model.parameters(),lr=1e-2)

আমরা এটিকে একটি `FederatedDataLoader` এ রেখেছি এবং অপশনগুলো নির্দিষ্ট করি

In [0]:
train_loader = sy.FederatedDataLoader(dataset, batch_size=32, shuffle=False, drop_last=False)

এবং অবশেষে আমরা epoch এর উপরে iterate করি। খাঁটি এবং স্থানীয় PyTorch প্রশিক্ষণের সাথে এটির তুলনা করলে এদের মধ্যমের মিল দেখতে পারবেন!

In [0]:
for epoch in range(1, epochs + 1):
    loss_accum = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        model.send(data.location)
        
        optimizer = optimizers[data.location.id]
        optimizer.zero_grad()
        pred = model(data)
        loss = ((pred.view(-1) - target)**2).mean()
        loss.backward()
        optimizer.step()
        
        model.get()
        loss = loss.get()
        
        loss_accum += float(loss)
        
        if batch_idx % 8 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tBatch loss: {:.6f}'.format(
                epoch, batch_idx, len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))            
            
    print('Total loss', loss_accum)


# অভিনন্দন !!! - সম্প্রদায় যোগদানের সময়!

এই নোটবুক টিউটোরিয়ালটি সম্পন্ন করার জন্য অভিনন্দন! আপনি যদি এটি উপভোগ করেন এবং গোপনীয়তা সংরক্ষণ, এআই এবং এআই সরবরাহ চেইনের (ডেটা) বিকেন্দ্রীভূত মালিকানার দিকে আন্দোলনে যোগ দিতে চান, আপনি নিম্নলিখিত উপায়ে এটি করতে পারেন!

### গিটহাবে স্টার পাইসাইফ্ট

আমাদের কমিউনিটিকে সাহায্য করার সবচেয়ে সহজ পন্থা হলো রিপোজিটোরিতে স্টার দেয়া! এটি আমরা যে দারুন সরঞ্জাম তৈরি করছি সে ব্যাপারে সচেতনতা বৃদ্ধি করতে সাহায্য করে।

- [Star PySyft](https://github.com/OpenMined/PySyft)

### আমাদের স্ল্যাকে(Slack) যোগ দিন!

সর্বশেষতম অগ্রগতিতে আপ টু ডেট রাখার সর্বোত্তম উপায় হলো আমাদের সম্প্রদায়ে যোগদান করা! আপনি [http://slack.openmined.org](http://slack.openmined.org) এ ফর্মটি পূরণ করে এটি করতে পারেন

### একটি কোড প্রকল্পে যোগদান করুন!

আমাদের কমিউনিটিতে অবদান রাখার সেরা উপায় হলো একজন কোড অবদানকারীতে পরিণত হওয়া। যেকোন সময় আপনি PySyft এর গিটহাব ইস্যুর পেজে যেতে পারেন এবং "Projects" দিয়ে বাছাই করবেন। এর মাধ্যমে আপনি যে সকল প্রজেক্টে যোগদান করতে পারবেন সেগুলোর উপরের দিকের টিকেটের ওভারভিউ পাবেন। আপনি যদি কোন প্রজেক্টে জয়েন করতে ইচ্ছুক না হোন, কিন্তু কিছুটা কোডিং করতে ইচ্ছুক সেক্ষেত্রে আপনি "one off" মিনি প্রজেক্টগুলো দেখতে পারেন গিটহাব ইস্যুতে "good first issue" চিহ্নিত ইস্যুগুলো।

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### দান করুন

আপনার যদি আমাদের কোডবেজে অবদান রাখারা সময় না হয়, কিন্তু এরপরও আমাদেরকে সমর্থন দিতে চান তাহলে আমাদের উন্মুক্ত সংগ্রহের সমর্থক হতে পারেন। সকল ধরনের দানের অর্থ আমাদের ওয়েব হোস্টিং এবং অন্যান্য কমিউনিটি কার্যক্রমে খরচ হয় যেমন - হ্যাকাথন, মিটাপ।

[OpenMined's Open Collective Page](https://opencollective.com/openmined)